### Simplest Python Program

This is the simplest program we can start with. 

It creates the aiml object, learns the startup file, and then loads the rest of the aiml files. 

After that, it is ready to chat, and we enter an infinite loop that will continue to prompt the user for a message. 

You will need to enter a pattern the bot recognizes. The patterns recognized depend on what AIML files you loaded.

We create the startup file as a separate entity so that we can add more aiml files to the bot later without having to modify any of the programs source code. We can just add more files to learn in the startup xml file.

import aiml

##### Create the kernel and learn AIML files
kernel = aiml.Kernel()
kernel.learn("std-startup.xml")
kernel.respond("load aiml b")

while True:
    print(kernel.respond(input("Enter your message >> ")))

### Speeding up Brain Load

When you start to have a lot of AIML files, it can take a long time to learn. This is where brain files come in. After the bot learns all the AIML files it can save its brain directly to a file which will drastically speed up load times on subsequent runs.

In [1]:
import aiml
import os
import sys

kernel = aiml.Kernel()

if os.path.isfile("bot_brain.brn"):
    kernel.bootstrap(brainFile = "bot_brain.brn")
else:
    kernel.bootstrap(learnFiles = "std-startup.xml", commands = "load aiml b")
    kernel.saveBrain("bot_brain.brn")

# kernel now ready for use

Loading brain from bot_brain.brn...done (4 categories in 0.00 seconds)
Kernel bootstrap completed in 0.00 seconds


In [2]:
while True:
    inp = input("Enter your message or bye to quit >> ")
    if inp.lower() == "bye": 
        print('\n Thanks for chatting! See you soon')
        break
    else: 
        print(kernel.respond(inp))
    

Enter your message or bye to quit >> Hello
Well, hello!
Enter your message or bye to quit >> Bye

 Thanks for chatting! See you soon


### Connect to a data base

In [ ]:
import   mysql.connector, re
import   pandas              as  pd
import   numpy               as  np

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="",
  database="matrimony"
)

mycursor = mydb.cursor()

### Get user input

In [4]:
Profile_ID = input("Enter your Profile ID>>")
print(Profile_ID)

Enter your Profile ID>>M5929711
M5929711


In [5]:
mycursor.execute("SELECT * FROM database_for_matrimony1 where PROFILE_ID = %s", (Profile_ID,))

myresult     = mycursor.fetchone()


mycol        = ['S_No','Profile_ID','Age','Gender','Marital_Status','Caste','Sub_Caste','State','City','Country','Height',\
                'physical_status','Family_Status','Family_Type','Family_Values','Groom_s_Age','Groom_s_Education',\
                'Groom_s_Income','Groom_s_Location','About_Description']     


df            = pd.DataFrame([myresult], columns= mycol, index = None)

print(df.shape)
print(df.head())  
print(list(df))
print(df.info())

(1, 20)
   S_No Profile_ID  Age  Gender Marital_Status     Caste  Sub_Caste  \
0     1   M5929711   27  Female  Never married  Mudaliar  Melakarar   

        State     City Country Height physical_status Family_Status  \
0  Tamil Nadu  Chennai   India    5.8          Normal  Middle Class   

  Family_Type Family_Values Groom_s_Age Groom_s_Education  Groom_s_Income  \
0       Joint      Orthodox       28-30          Graduate          400000   

  Groom_s_Location About_Description  
0       Tamil Nadu                NA  
['S_No', 'Profile_ID', 'Age', 'Gender', 'Marital_Status', 'Caste', 'Sub_Caste', 'State', 'City', 'Country', 'Height', 'physical_status', 'Family_Status', 'Family_Type', 'Family_Values', 'Groom_s_Age', 'Groom_s_Education', 'Groom_s_Income', 'Groom_s_Location', 'About_Description']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 20 columns):
S_No                 1 non-null int64
Profile_ID           1 non-null object
Age           

### Set session variables 

In [6]:
sessionId     =  Profile_ID
sessionData   =  kernel.getSessionData(sessionId)

kernel.setPredicate("Age",              df.Age[0],            sessionId)
kernel.setPredicate("Gender",           df.Gender[0],         sessionId)
kernel.setPredicate("Marital_Status",   df.Marital_Status[0], sessionId)
kernel.setPredicate("Partner_s_Age",    df.Groom_s_Age[0],    sessionId)


### Get session variables 

In [7]:
client_age              =   kernel.getPredicate("Age", sessionId)
client_gender           =   kernel.getPredicate("Gender", sessionId)
client_Marital_Status   =   kernel.getPredicate("Marital_Status", sessionId)
client_Partner_s_Age    =   kernel.getPredicate("Partner_s_Age", sessionId)

In [8]:
print("\n Your profile ID is %s Age is %d Gender is %s Marital Status is %s "%(sessionId, client_age, client_gender, client_Marital_Status))
print("\n You prefer partners in the age range  %s" %(client_Partner_s_Age))


 Your profile ID is M5929711 Age is 27 Gender is Female Marital Status is Never married 

 You prefer partners in the age range  28-30


In [9]:
lo, up = re.findall('\d+', client_Partner_s_Age)

lo     = int(lo)
up     = int(up)
print(lo, up)
print(type(lo))
print(type(up))    

28 30
<class 'int'>
<class 'int'>


### Forward matches - 

In [10]:
mycursor.execute("SELECT count(*) FROM database_for_matrimony1 where Gender = 'Male' AND Age <= %s AND  Age >= %s", (up, lo,))

myresult     = mycursor.fetchone()
print(myresult)

(19,)


### Relax Age limit and check the match count

In [11]:
print('\n You have set your partner age limit as %s' % client_Partner_s_Age)
ans = input("Do you want to relax age limit? Y/N>>")
print(ans)

if ans.lower() == 'N': sys.exit(0)

relax_years = input("By how many years, do you want to relax your partner upper age limit? >>")
print(relax_years)


 You have set your partner age limit as 28-30
Do you want to relax age limit? Y/N>>Y
Y
By how many years, do you want to relax your partner upper age limit? >>3
3


In [12]:
age_l          = 0
age_u          = 0
age_l          = up + 1
age_u          = up + int(relax_years) 

print('\nRelaxed upper age limit is now %d - %d' %(age_l, age_u))
relaxed_age_limit = range(age_l, age_u + 1)

for age_i in relaxed_age_limit:
    mycursor.execute("SELECT count(*) FROM database_for_matrimony1 where Gender = 'Male' AND Age = %s ", (age_i,))
    myresult     = mycursor.fetchone()
    print('\nAt age %d, you will get %s more matches from male partners' % (age_i, myresult[0]))


Relaxed upper age limit is now 31 - 33

At age 31, you will get 6 more matches from male partners

At age 32, you will get 1 more matches from male partners

At age 33, you will get 0 more matches from male partners


### Reverse matches

In [13]:
Qry = "SELECT count(*) FROM database_for_matrimony1 where Gender = 'Male' AND Age <= %s AND  Age >= %s "
Qry = Qry + "AND substring(Groom_s_Age,1,instr(Groom_s_Age,'-')-1) <= %s AND substring(Groom_s_Age,instr(Groom_s_Age,'-')+1) >= %s "

print(Qry)
mycursor.execute(Qry, (up, lo, client_age, client_age,))
myresult     = mycursor.fetchone()
print(myresult)

SELECT count(*) FROM database_for_matrimony1 where Gender = 'Male' AND Age <= %s AND  Age >= %s AND substring(Groom_s_Age,1,instr(Groom_s_Age,'-')-1) <= %s AND substring(Groom_s_Age,instr(Groom_s_Age,'-')+1) >= %s 


ProgrammingError: Failed processing format-parameters; Python 'int64' cannot be converted to a MySQL type

### END